In [1]:
#Dependencies
from openpyxl import load_workbook
import openpyxl as pyxl
from pathlib import Path
import os
from datetime import datetime

In [2]:
#This cell can be used for debugging to confirm whether or not the path you've entered is valid. 
Path(r"C:\Invoices").exists()

True

In [3]:
subitems=[] #Will contain the data we're aggregating for all items on invoices
invoiceTotal=[] #Will contain the final data for each invoice
fls=[] #Store file paths for excel files to be searched

In [15]:
#Option 1: Going to only specified files

In [16]:
#Helper function to build path strings when iterating through multiple files
def pthBld(flNm,flExtn="",path_building_variable=('Paid',0)):
    """Take a file name and expected file type, and based on some other input, add the rest of the file path for reference by workbook opening function"""
    if flExtn!="": flExtn="."+flExtn
    tp=path_building_variable[0]
    yr=path_building_variable[1]
    if yr==22:
        return (r"C:\Invoices\Invoices 2022"+chr(92)+tp+chr(92)+flNm+flExtn)
    else:
        return (r"C:\Invoices\Invoices '"+str(yr)+chr(92)+tp+chr(92)+flNm+flExtn)

In [17]:
#Test function. Output will have double slashes due to vagaries of paths/strings in this language etc but if exists() yields True, the path is good
p=pthBld('Invoice 3',"xlsx",('unPaid',22))
p, "Exists?: "+str(Path(p).exists())

('C:\\Invoices\\Invoices 2022\\unPaid\\Invoice 3.xlsx', 'Exists?: True')

In [99]:
fls.append(pthBld('Invoice 1',"xlsx",('Paid',22)))
fls.append(pthBld('Invoice 2',"xlsx",('Paid',22)))
fls.append(pthBld('Invoice 3',"xlsx",('unPaid',22)))
fls.append(pthBld('Invoice 1',"xlsx",('Paid',21)))
fls.append(pthBld('Invoice 2',"xlsx",('Paid',21)))
fls.append(pthBld('Invoice 3',"xlsx",('unPaid',21)))

In [8]:
#Option 2: Searching for every file in a given directory based on name structure

In [4]:
def list_dir(directory):
    """This function will print out the path for every file in the specified directory (and any further nested directories)"""
    output=[] #initialize
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            output.append(f)
        elif os.path.isdir(f):
            output.extend(list_dir(f))
    return output

In [5]:
list_dir(r"C:\Invoices") #Test it works

["C:\\Invoices\\Invoices '21\\junk_Data.xlsx",
 "C:\\Invoices\\Invoices '21\\Paid\\invoice 1.xlsx",
 "C:\\Invoices\\Invoices '21\\Paid\\invoice 2.xlsx",
 "C:\\Invoices\\Invoices '21\\Unpaid\\invoice 3.xlsx",
 "C:\\Invoices\\Invoices '23\\invoice 1.xlsx",
 "C:\\Invoices\\Invoices '23\\invoice 2.xlsx",
 "C:\\Invoices\\Invoices '23\\invoice 3.xlsx",
 "C:\\Invoices\\Invoices '23\\junk_Data.xlsx",
 'C:\\Invoices\\Invoices 2022\\junk_Data.xlsx',
 'C:\\Invoices\\Invoices 2022\\Paid\\invoice 1.xlsx',
 'C:\\Invoices\\Invoices 2022\\Paid\\invoice 2.xlsx',
 'C:\\Invoices\\Invoices 2022\\Paid\\junk_Data.xlsx',
 'C:\\Invoices\\Invoices 2022\\Unpaid\\invoice 3.xlsx']

In [6]:
#Specify a search criteria the file name should meet for us to apply our method to it:
#We will simply want "invoice" to be in the name.. do not get the 'junk data' files
for f in list_dir(r"C:\Invoices"):
    if "invoice" in f.lower()[-15:]: 
        fls.append(f)

In [7]:
fls #Check it worked

["C:\\Invoices\\Invoices '21\\Paid\\invoice 1.xlsx",
 "C:\\Invoices\\Invoices '21\\Paid\\invoice 2.xlsx",
 "C:\\Invoices\\Invoices '21\\Unpaid\\invoice 3.xlsx",
 "C:\\Invoices\\Invoices '23\\invoice 1.xlsx",
 "C:\\Invoices\\Invoices '23\\invoice 2.xlsx",
 "C:\\Invoices\\Invoices '23\\invoice 3.xlsx",
 'C:\\Invoices\\Invoices 2022\\Paid\\invoice 1.xlsx',
 'C:\\Invoices\\Invoices 2022\\Paid\\invoice 2.xlsx',
 'C:\\Invoices\\Invoices 2022\\Unpaid\\invoice 3.xlsx']

In [8]:
#Search the files gotten from option 1 or 2
#Knowing our file structure we will go through column C from the first item in each invoice until no more items
#are observed, and record relevant info from the record.
#Then make a second data store with the bottom line info for the invoice 
for fl in fls:
    wb = load_workbook(filename = fl,data_only=True)
    #First record data for the sub-items list
    print(str(1+fls.index(fl))+chr(92)+str(len(fls)),fl)
    for i in range(8,30): #Note arbitrary assumption on row limit. Starting point from existing sheet structure
        #Grab invoice admin data
        if wb['Commercial Invoice'].cell(row=i,column=3).value!=None:
            data=[fl] #include filepath in data dump
            data.append(wb['Commercial Invoice'].cell(row=3,column=8).value) # Invoice number to data dump
            data.extend([wb['Commercial Invoice'].cell(row=i,column=c).value if type(wb['Commercial Invoice'].cell(row=i,column=c).value)!=datetime else str(wb['Commercial Invoice'].cell(row=i,column=c).value) for c in range(2,9)]) #Convert any data to string.. when re printed to excel it will cast to appropriate type
            subitems.append(data)
        else:
            pass
    #Now record to totals
    data2=[fl] #initialize with filepath
    data2.append(str(wb['Commercial Invoice'].cell(row=4,column=8).value)) #invoice data
    data2.append(wb['Commercial Invoice'].cell(row=3,column=8).value) #invoice num
    data2.append(wb['Commercial Invoice'].cell(row=13,column=8).value) #subtotal
    data2.append(wb['Commercial Invoice'].cell(row=14,column=8).value) #VAT
    data2.append(wb['Commercial Invoice'].cell(row=15,column=8).value) #Other Tax
    data2.append(wb['Commercial Invoice'].cell(row=16,column=8).value) #Delivery
    data2.append(wb['Commercial Invoice'].cell(row=17,column=8).value) #Deposit Recieved
    data2.append(wb['Commercial Invoice'].cell(row=18,column=8).value) #Total
    invoiceTotal.append(data2)

C:\ProgramData\Anaconda\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


1\9 C:\Invoices\Invoices '21\Paid\invoice 1.xlsx
2\9 C:\Invoices\Invoices '21\Paid\invoice 2.xlsx
3\9 C:\Invoices\Invoices '21\Unpaid\invoice 3.xlsx
4\9 C:\Invoices\Invoices '23\invoice 1.xlsx
5\9 C:\Invoices\Invoices '23\invoice 2.xlsx
6\9 C:\Invoices\Invoices '23\invoice 3.xlsx
7\9 C:\Invoices\Invoices 2022\Paid\invoice 1.xlsx
8\9 C:\Invoices\Invoices 2022\Paid\invoice 2.xlsx
9\9 C:\Invoices\Invoices 2022\Unpaid\invoice 3.xlsx


In [9]:
#Check we got what we wanted
len(subitems),subitems[0], len(invoiceTotal),invoiceTotal[0]

(9,
 ["C:\\Invoices\\Invoices '21\\Paid\\invoice 1.xlsx",
  34567,
  '2023-03-12 00:00:00',
  789807,
  'Wooden Blocks',
  6,
  7,
  1.5,
  40.5],
 9,
 ["C:\\Invoices\\Invoices '21\\Paid\\invoice 1.xlsx",
  '2023-03-12 00:00:00',
  34567,
  40.5,
  0.089,
  3.6045,
  5,
  0,
  49.1045])

In [10]:
#Finally, print our aggregated data to a single subitems table
wb = pyxl.Workbook()
ws = wb.active
r=0 
for row in subitems:
    r=r+1
    c=0
    for index in range(len(row)):
        c=c+1
        ws.cell(row=r,column=c).value=row[index]
wb.create_sheet('Bottom Line Info')
ws=wb['Bottom Line Info']
r=0 
for row in invoiceTotal:
    r=r+1
    c=0
    for index in range(len(row)):
        c=c+1
        ws.cell(row=r,column=c).value=row[index]
wb.save(r"C:\Invoices\Aggregated_Data.xlsx")
wb.close()